<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/November_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install firebase-admin 

In [0]:
import firebase_admin
from firebase_admin import credentials, db, storage 
import cv2 
import math 
import numpy as np
import matplotlib.pyplot as plt 
import csv 
import pandas as pd

In [0]:

cred = credentials.Certificate("/content/celestinihdr-2-firebase-adminsdk-x7gew-beb69daef0.json")
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'celestinihdr-2.appspot.com',
}, name='novemeber')

In [0]:
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

anandvihar_hdr=[]
wazirpur_hdr=[]
wazirpur_jpeg={}
ctr=0
for blob in blobs:
  if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[27]=='j' and int(blob.name[43:45])<18 and int(blob.name[43:45])>6):
    blob.download_to_filename('/content/' + "wazirpur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg')
    name = "wazirpur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg' 
    ctr+=1
    print(ctr)
    im = cv2.imread(name)
    im = cv2.resize(im,(256,256))
    wazirpur_jpeg[blob.name]=im
  # if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[27]=='h' and int(blob.name[42:44])<18 and int(blob.name[42:44])>6): 
  #   blob.download_to_filename('/content/' + "wazirpur_hdr" + str(blob.name.split('/')[-1])+ '.jpeg')
  #   name = "wazirpur_hdr" + str(blob.name.split('/')[-1])+ '.jpeg' 
  #   ctr+=1
  #   print(ctr)
  #   im = cv2.imread(name)
  #   im = cv2.resize(im,(256,256))
  #   wazirpur_hdr.append(im)
  # if(blob.name[7:13]=='static' and blob.name[14]=='H' and blob.name[21]=='h'):
  #    blob.download_to_filename('/content/' + "anandvihar_hdr" + str(blob.name.split('/')[-1])+ '.jpeg')
  #    name = "anandvihar_hdr" + str(blob.name.split('/')[-1])+ '.jpeg' 
  #    ctr+=1
  #    print(ctr)
  #    im = cv2.imread(name)
  #    im = cv2.resize(im,(256,256))
  #    anandvihar_hdr.append(im)
     
    
    

In [24]:
for i in wazirpur_jpeg:
  print(i)
  break

images/static/Punjabi_Bagh/jpeg/2019-11-12 07:43:39_Shadipur, New Delhi - CPCB


In [5]:
#print(len(wazirpur_hdr))
print(len(wazirpur_jpeg))
#print(len(anandvihar_hdr))

287


IMAGE PARAMETERS 

In [0]:
def trans_deg(im):
  B=np.zeros((256,256))
  D=np.zeros((256,256))
  C=np.zeros((256,256))
  # 3rd channel or im[2] has been split as b,g,r 

  b,g,r = cv2.split(im)

  for i in range(b.shape[0]):
    for j in range(b.shape[1]):
      D[i][j] = min(b[i][j], g[i][j], r[i][j])
      B[i][j] = max(b[i][j], g[i][j], r[i][j])
      C[i][j] = B[i][j] - D[i][j]

  d = np.sum(D)/float(np.size(D))
  b = np.sum(B)/float(np.size(B))
  c = b-d 
  A = (0.33)*max(np.ndarray.flatten(B)) + (0.66)*b
  
  x1=((A - d)/float(A)) 
  x2=(c/float(A))
  u = 10.127489
  v= -8.336512
  s = 0.13606234
  
  w = math.exp(-0.5*(u*x1 + v*x2)+s)
  
  if(w<=0.1):
      w=0
  elif(w<=0.3):
    w=1
  elif(w<=0.5):
    w=2
  elif(w<=0.7):
    w=3
  elif(w<=0.8):
    w=4
  elif(w<=0.9):
    w=5
    
  return w

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [107]:
feat=[]
ctr=0
for i in wazirpur_jpeg:
  l=[]
  l.append(i[32:48])
  l.append(entropy(wazirpur_jpeg[i]))
  l.append(contrast(wazirpur_jpeg[i]))
  l.append(trans_deg(wazirpur_jpeg[i]))
  feat.append(l)
  ctr+=1
  print(ctr)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: ignored

In [0]:
#IMAGE NAME MODIFICATION - for label matching 

image_match={}
for l in feat:
  temp = l[0]
  #print(temp)
  temp=temp[-24:-1]
  #print(temp)
  temp=temp.split('-')
  #print(temp)
  newtemp=temp[2].split(' ')
  #print(newtemp)
  newesttemp=newtemp[1].split(':')
  #print(newesttemp)
  j=int(newesttemp[1])
  if(j<=7.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<=15 or j<=22.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<=30 or j<=37.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<=45 or j<=52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '45'
  elif(j>52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  t=[]
  t.append(final[11:13])
  t.append(l[1])
  t.append(l[2])
  t.append(l[3])
  image_match[final]=t

  

WEATHER PARAMETERS 

In [0]:
cred = credentials.Certificate('/content/credentials_weather.json')

# The weather data after 1st July 
# Initialize the app with a service account, granting admin privileges
new = firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://celestini-bd493.firebaseio.com/'
}, name ='w')

In [0]:

ref = db.reference(app=new)
df = ref.get()


In [0]:
temp_weather = {}

for key in df:
  if(key[0:10]=='12-11-2019'):
    l=[]
    l.append(df[key]['main']['humidity'])
    l.append(df[key]['main']['pressure'])
    l.append(df[key]['main']['temp'])
    l.append(df[key]['wind']['deg'])
    l.append(df[key]['wind']['speed'])
    temp_weather[key]=l
      

LABELS 

In [0]:
df_ = pd.read_csv('/content/november_wazirpur.csv')
data_ = df_.mask(df_.eq('None')).dropna()
file_ = data_.to_csv('/content/wazirpur_labels_new.csv')

label =[]
k=0
with open('/content/wazirpur_labels_new.csv') as file:
  reader = csv.reader(file, delimiter =',')
  for row in reader:
    k+=1
    if(k>3):
      l=[]
      l.append(row[1]) #DATE 
      l.append(row[3]) #PM
      label.append(l) 

In [0]:
#WEATHER TMP MODIFICATION 
weather_match={}
for i in temp_weather:
  temp = i
  l=[]
  temp=temp[-24:-3]
  temp=temp.split('-')
  newtemp=temp[2].split(' ')
  newesttemp=newtemp[1].split(':')
  j=int(newesttemp[1])
  if(j<=7.5):
    final = temp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<=15 or j<=22.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<=30 or j<=37.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<=45 or j<=52.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '45'
  elif(j>52.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '00'
  weather_match[final]=temp_weather[i]
  
  
  

In [123]:
#GETTING THE PREVIOUS LABEL 

ctr=0
for l in label:
  ctr+=1
  print(l[1])
  print(label[ctr-4][1])
  break
  

216
401


In [117]:
ctr=0
for i in image_match:
  for l in label:
    if(l[0]==i):
      print(l)
      print(image_match[i])
      ctr+=1


['12-11-2019 07:00', '721']
['07', 39.154083251953125, 74.91918491287663, 1]
['12-11-2019 08:00', '697']
['08', 22.566495895385742, 44.9361855178027, 2]


92


In [0]:
for i in image_match:
  for l 
